In [ ]:
!pip install tensorflow==2.1.1

In [ ]:
import os 
os.mkdir("./model")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import pickle 
import numpy as np
image_size = (150,150)
X_train = np.array(pickle.load(open("/content/drive/MyDrive/IMC/X_train.pickle", "rb")))
y_train = np.array(pickle.load(open("/content/drive/MyDrive/IMC/y_train.pickle","rb")))
X_val = np.array(pickle.load(open("/content/drive/MyDrive/IMC/X_val.pickle","rb")))
y_val = np.array(pickle.load(open("/content/drive/MyDrive/IMC/y_val.pickle","rb")))


from tensorflow.keras.utils import to_categorical   
y_train = to_categorical(y_train, num_classes = 6)
y_val = to_categorical(y_val, num_classes=6)
y_val, y_train = np.array(y_val), np.array(y_train) 

# X_train = X_train[0]
# y_train = y_train[0]
X_train.shape

(1440, 150, 150, 1)

In [ ]:
print(X_train.shape)
X_val.shape
y_val.shape
y_train.shape

(1440, 150, 150, 1)


(1440, 6)

In [ ]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Activation, Convolution2D, Dropout, Conv2D
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import glorot_uniform
import pandas as pd
import cv2
import numpy as np
import tensorflow.keras.optimizers
 
# parameters
batch_size = 32
num_epochs = 500
input_shape = (150, 150, 1)
verbose = 1
num_classes = 6
patience = 50
base_path = 'model/'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
 
# model parameters
regularization = l2(l2_regularization)

 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
 
# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same', kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same', kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same', kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same', kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)

# x = Flatten()(x)
# x = Dense(1024, activation = 'relu',)(x)
output = Activation('softmax',name='predictions')(x)
 
model = Model(img_input, output)
adam = tensorflow.keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.95, nesterov=False,)
adagrad = tf.keras.optimizers.Adagrad()
model.compile(optimizer= adam, loss='categorical_crossentropy',metrics=['accuracy'])
# model.summary()

 
# callbacks
log_file_path = base_path + '_imgclass.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience= 10, verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop,reduce_lr]

# model.summary() 
history = model.fit(data_generator.flow(X_train, y_train,batch_size = batch_size),
                    steps_per_epoch = len(X_train)// batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data= (X_val,y_val))


In [ ]:
# import keras
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [76]:
import pandas as pd
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import os
# from tqdm.tqdm import tqdm

img_size = 150
dirpath = "/content/drive/MyDrive/IMC/Testing/"
model_path = '/content/_mini_XCEPTION.24-0.49.hdf5'
labels = ["Bicycle", "Boat", "Cat", "Motorbike", "People","Table"]

hot = []
predictions = []
df = pd.read_csv("/content/testnames.csv")
df.head()
img_classifier = load_model(model_path, compile=True)
# img_array = cv2.imread('/content/2015_06720.jpg',cv2.IMREAD_GRAYSCALE)
# new_array = cv2.resize(img_array, (img_size, img_size,))
# new_array = np.array(new_array).reshape(1,img_size, img_size, 1)
# plt.imshow(img_array)
# # new_array = np.array(new_array).reshape(img_size, img_size, 1)
# preds = img_classifier.predict(new_array)
# probs = np.max(preds)
# label = labels[preds.argmax()]
# predictions.append(label)
for i in tqdm(df['name']):
    img_array = cv2.imread(os.path.join(dirpath,i),cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (img_size, img_size,))
    new_array = np.array(new_array).reshape(1,img_size, img_size, 1)
    preds = img_classifier.predict(new_array)
    probs = np.max(preds)
    label = labels[preds.argmax()]
    predictions.append(label)
    hot.append(probs)

100%|██████████| 480/480 [00:35<00:00, 13.41it/s]


In [ ]:
from collections import Counter
Counter(predictions)

In [49]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
actual = []
for i in y_val:
  actual.append(labels[i.argmax()])
confusion_matrix(actual, predictions)

In [51]:
score = img_classifier.evaluate(X_val,y_val, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 1.4778087139129639
Test accuracy: 0.4895833432674408


In [98]:
df = pd.read_csv("/content/testnames.csv")
df['class'] = predictions
df = df.iloc[:,1:]
df.head(50)
# df.iloc[:,0]

,name,class
0,2015_00401.jpg,Table
1,2015_00402.jpg,Boat
2,2015_00403.jpg,Table
3,2015_00404.jpg,Boat
4,2015_00405.jpg,Boat
5,2015_00406.jpg,Boat
6,2015_00407.jpg,Boat
7,2015_00408.jpg,Boat
8,2015_00409.jpg,Table
9,2015_00410.jpg,Table


In [99]:
df.to_csv('testoutput.csv',)